In [6]:
import boto3
import time
import sagemaker
import fitz
from io import BytesIO

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [7]:
from textractor import Textractor
from textractor.data.constants import Textract_API, TextractFeatures
from textractor.data.text_linearization_config import TextLinearizationConfig

In [8]:
filetexts  = dict()

In [9]:
def is_pdf_openable(bucket,key):
    try:
        response=s3_client.get_object(Bucket=bucket,Key=key)
        pdf_content=response['Body'].read()
        doc=fitz.open(stream=BytesIO(pdf_content))
        doc.close()
        return True
    except Exception as e:
        print(f"error opening {key}: {str(e)}")
        return False

In [13]:
s3_client = boto3.client('s3')
s3_bucket_name = 's3sagemakerbucket'
count=0
pdf_files=[]
def read_pdf_files_from_folder(folder_path):
    global count
    response = s3_client.list_objects_v2(Bucket=s3_bucket_name, Prefix=folder_path)
    for obj in response['Contents']:
        if obj['Key'].endswith('.pdf'):
            print(obj['Key'])
            if is_pdf_openable(s3_bucket_name,obj['Key']):
                count=count+1
                pdf_files.append(obj['Key'])

                document_name = obj['Key']
                file=obj['Key'].split('/')[-1]

                extractor = Textractor(region_name="eu-west-1")
                markdown_text = ""
                textract_json = extractor.start_document_analysis(
                    file_source=f"s3://s3sagemakerbucket/{obj['Key']}",
                    features=[TextractFeatures.LAYOUT, TextractFeatures.TABLES],
                    save_image=False,
                )
                response_textract_async = extractor.get_result(job_id=textract_json.job_id, api=Textract_API.ANALYZE)
                markdown_text = response_textract_async.to_markdown()
                filetexts[file] = markdown_text
                print(f"files completed: [{count}/65]")
        else:
            if obj['Key'].endswith('\/'):
                read_pdf_files_from_folder(obj['Key'])

folder_name = "BondingLine_data/"
read_pdf_files_from_folder(folder_name)
print(count)

BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2006.pdf
files completed: [1/65]
BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2014.pdf
files completed: [2/65]
BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2016.pdf
files completed: [3/65]
BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2018.pdf
files completed: [4/65]
BondingLine_data/Identification 2/DACH/SE Gemany_Commerzbank_amendment nr._6_11.2016.pdf
files completed: [5/65]
BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_2022.pdf
files completed: [6/65]
BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment1_2017.pdf
files completed: [7/65]
BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment2_2018.pdf
files completed: [8/65]
BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment3_2019.pdf
files completed: [9/65]
BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment4_2019.pdf
files co

In [14]:
len(filetexts)

101

In [15]:
print(pdf_files)

['BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2006.pdf', 'BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2014.pdf', 'BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2016.pdf', 'BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2018.pdf', 'BondingLine_data/Identification 2/DACH/SE Gemany_Commerzbank_amendment nr._6_11.2016.pdf', 'BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_2022.pdf', 'BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment1_2017.pdf', 'BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment2_2018.pdf', 'BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment3_2019.pdf', 'BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment4_2019.pdf', 'BondingLine_data/Identification 2/DACH/SE Gemany_Nordea_2016.pdf', 'BondingLine_data/Identification 2/DACH/SE Germany_Deutsche Bank_Areva.pdf', 'BondingLine_data/Identification 2/DACH/SE Switz

In [24]:
!mkdir -p "bonding_line_textfiles"
for file, text in filetexts.items():
    file2 = file.split('.')[0]
    file2 = file.replace('.pdf','').replace('.','')
    with open(f"bonding_line_textfiles/{file2}.txt", 'w', encoding = "utf-8") as outfile:
        outfile.write(text)
    outfile.close()